# Querying Data From AWS Redshift

## Connecting with the database

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%config IPCompleter.greedy=True

In [5]:
import sys
reload(sys)
sys.setdefaultencoding('utf8')

In [6]:
import sqlalchemy
import psycopg2
import simplejson
%load_ext sql
%config SqlMagic.displaylimit = 5

In [4]:
# Read the Redshift's credentials file 
with open("redshift_creds.json.nogit") as fh:
    creds = simplejson.loads(fh.read())

# Connect to the Redshift
connect_to_db = 'postgresql+psycopg2://' + \
                creds['user_name'] + ':' + creds['password'] + '@' + \
                creds['host_name'] + ':' + creds['port_num'] + '/' + creds['db_name'];
%sql $connect_to_db

u'Connected: dwahid@data_depot'

## Fraud Accounts Acitivies: Lifespan, Invoice and Client Count in Different Time Periods

In [8]:
sql_fraud_accounts_invoices_clients_activity ='''WITH fraud_systems AS ( -- THIS IS A FILTER
    SELECT systemid FROM data_science.fraud_accounts_list GROUP BY 1
), fraud_status_date AS (
    SELECT systemid,
           status_date AS last_date
    FROM data_science.fraud_accounts_list
    JOIN fraud_systems USING(systemid)
),  client_dates AS (
    SELECT
        u.systemid,
        u.userid,
        u.signup_date,
        DATEDIFF(days, rs.signup_date, u.signup_date)  AS days_to_client_creation
    from coalesced_live_shards."user" AS u
    LEFT JOIN report_systems rs USING(systemid)
    WHERE u.level = 0
),  client_groupings AS (
    SELECT
        systemid,
        SUM(CASE WHEN days_to_client_creation BETWEEN 0 AND 8 THEN 1 ELSE 0 END) AS client_count_day_7,
        SUM(case when days_to_client_creation BETWEEN  0 AND 16 THEN 1 ELSE 0 END) AS client_count_day_15,
        SUM(case when days_to_client_creation BETWEEN  0 AND 31 THEN 1 ELSE 0 END) AS client_count_day_30,
        SUM(case when days_to_client_creation BETWEEN  0 AND 46 THEN 1 ELSE 0 END) AS client_count_day_45,
        SUM(case when days_to_client_creation BETWEEN  0 AND 61 THEN 1 ELSE 0 END) AS client_count_day_60,
        SUM(case when days_to_client_creation BETWEEN  0 AND 76 THEN 1 ELSE 0 END) AS client_count_day_75,
        SUM(case when days_to_client_creation BETWEEN 0 AND 91 THEN 1 ELSE 0 END) AS client_count_day_90,
        SUM(case when days_to_client_creation BETWEEN 0 AND 181 THEN 1 ELSE 0 END) AS client_count_month_6,
        SUM(case when days_to_client_creation BETWEEN 0 AND 366 THEN 1 ELSE 0 END) AS client_count_year_1,
        SUM(case when days_to_client_creation BETWEEN 0 AND 731 THEN 1 ELSE 0 END) AS client_count_year_2,
        SUM(case when days_to_client_creation BETWEEN 0 AND 1096 THEN 1 ELSE 0 END) AS client_count_year_3,
        SUM(case when days_to_client_creation BETWEEN 0 AND 1461 THEN 1 ELSE 0 END) AS client_count_year_4,
        SUM(case when days_to_client_creation BETWEEN 0 AND (5*365+1) THEN 1 ELSE 0 END) AS client_count_year_5,
        SUM(case when days_to_client_creation BETWEEN 0 AND (6*365+1) THEN 1 ELSE 0 END) AS client_count_year_6,
        SUM(case when days_to_client_creation BETWEEN 0 AND(7*365+1) THEN 1 ELSE 0 END) AS client_count_year_7,
        SUM(case when days_to_client_creation BETWEEN 0 AND (8*365+1) THEN 1 ELSE 0 END) AS client_count_year_8,
        SUM(case when days_to_client_creation BETWEEN 0 AND (9*365+1) THEN 1 ELSE 0 END) AS client_count_year_9,
        SUM(case when days_to_client_creation BETWEEN 0 AND (10*365+1) THEN 1 ELSE 0 END) AS client_count_year_10
    FROM client_dates
    GROUP BY 1
), invoice_dates AS (
    SELECT
           inv.systemid,
           inv.invoiceid,
           inv.create_date,
           DATEDIFF(DAYS, rs.signup_date, inv.create_date) AS days_to_invoice_creation
    FROM coalesced_live_shards.invoice AS inv
             LEFT JOIN report_systems rs USING (systemid)
    WHERE inv.active = 1
), invoice_groupings AS (
    SELECT
        systemid,
        COUNT(invoiceid) AS invoice_count,
        SUM(CASE WHEN days_to_invoice_creation BETWEEN 0 AND 8 THEN 1 ELSE 0 END) AS invoice_count_day_7,
        sum(case when days_to_invoice_creation between 0 and 16 THEN 1 ELSE 0 END) AS invoice_count_day_15,
        sum(case when days_to_invoice_creation between 0 and 31 THEN 1 ELSE 0 END) AS invoice_count_day_30,
        sum(case when days_to_invoice_creation between 0 and 46 THEN 1 ELSE 0 END) AS invoice_count_day_45,
        sum(case when days_to_invoice_creation between 0 and 61 THEN 1 ELSE 0 END) AS invoice_count_day_60,
        sum(case when days_to_invoice_creation between 0 and 76 THEN 1 ELSE 0 END) AS invoice_count_day_75,
        sum(case when days_to_invoice_creation between 0 and 91 THEN 1 ELSE 0 END) AS invoice_count_day_90,
        sum(case when days_to_invoice_creation between 0 and 181 THEN 1 ELSE 0 END) AS invoice_count_month_6,
        sum(case when days_to_invoice_creation between 0 and (1*365+1) THEN 1 ELSE 0 END) AS invoice_count_year_1,
        sum(case when days_to_invoice_creation between 0 and (2*365+1) THEN 1 ELSE 0 END) AS invoice_count_year_2,
        sum(case when days_to_invoice_creation between 0 and (3*365+1) THEN 1 ELSE 0 END) AS invoice_count_year_3,
        sum(case when days_to_invoice_creation between 0 and (4*365+1) THEN 1 ELSE 0 END) AS invoice_count_year_4,
        sum(case when days_to_invoice_creation between 0 and (5*365+1) THEN 1 ELSE 0 END) AS invoice_count_year_5,
        sum(case when days_to_invoice_creation between 0 and (6*365+1) THEN 1 ELSE 0 END) AS invoice_count_year_6,
        sum(case when days_to_invoice_creation between 0 and (7*365+1) THEN 1 ELSE 0 END) AS invoice_count_year_7,
        sum(case when days_to_invoice_creation between 0 and (8*365+1) THEN 1 ELSE 0 END) AS invoice_count_year_8,
        sum(case when days_to_invoice_creation between 0 and (9*365+1) THEN 1 ELSE 0 END) AS invoice_count_year_9,
        SUM(case when days_to_invoice_creation between 0 and (10*365+1) THEN 1 ELSE 0 END) AS invoice_count_year_10
    FROM invoice_dates
    GROUP BY 1
)
SELECT
    a.systemid ,
    a.client_count,
    inv.invoice_count,
    signup_date,
    fr.last_date::DATE AS last_date,
    DATEDIFF(DAY, signup_date, fr.last_date::DATE) AS days_to_ban_hammer,
    c.client_count_day_7,
    c.client_count_day_15,
    c.client_count_day_30,
    c.client_count_day_45,
    c.client_count_day_60,
    c.client_count_day_75,
    c.client_count_day_90,
    c.client_count_month_6,
    c.client_count_year_1,
    c.client_count_year_2,
    c.client_count_year_3,
    c.client_count_year_4,
    c.client_count_year_5,
    c.client_count_year_6,
    c.client_count_year_7,
    c.client_count_year_8,
    c.client_count_year_9,
    c.client_count_year_10,
    inv.invoice_count_day_7,
    inv.invoice_count_day_15,
    inv.invoice_count_day_30,
    inv.invoice_count_day_45,
    inv.invoice_count_day_60,
    inv.invoice_count_day_75,
    inv.invoice_count_day_90,
    inv.invoice_count_month_6,
    inv.invoice_count_year_1,
    inv.invoice_count_year_2,
    inv.invoice_count_year_3,
    inv.invoice_count_year_4,
    inv.invoice_count_year_5,
    inv.invoice_count_year_6,
    inv.invoice_count_year_7,
    inv.invoice_count_year_8,
    inv.invoice_count_year_9,
    inv.invoice_count_year_10
FROM report_systems a

JOIN fraud_status_date AS fr USING(systemid)
LEFT JOIN client_groupings c USING(systemid)
LEFT JOIN invoice_groupings inv USING (systemid)
ORDER BY days_to_ban_hammer DESC;'''


df_sql_fraud_accounts_invoices_clients_activity = pd.read_sql_query(sql_fraud_accounts_invoices_clients_activity, connect_to_db)

In [9]:
df_sql_fraud_accounts_invoices_clients_activity.tail()

,systemid,client_count,invoice_count,signup_date,last_date,days_to_ban_hammer,client_count_day_7,client_count_day_15,client_count_day_30,client_count_day_45,...,invoice_count_year_1,invoice_count_year_2,invoice_count_year_3,invoice_count_year_4,invoice_count_year_5,invoice_count_year_6,invoice_count_year_7,invoice_count_year_8,invoice_count_year_9,invoice_count_year_10
488,3162278,0,NaN,2018-01-11,2018-01-12,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
489,3120710,1,NaN,2017-11-29,2017-11-30,1,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
490,4164112,2,3.0,2019-03-19,2019-03-20,1,2.0,2.0,2.0,2.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
491,4319694,1,NaN,2019-05-22,2019-05-23,1,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
492,4149206,0,NaN,2019-03-13,2019-03-13,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Export as csv file
df_sql_fraud_accounts_invoices_clients_activity.to_csv("/Users/dwahid/Documents/GitHub/fraud_detection/data/fraud_accounts_invoices_clients_activity.csv", 
                                      sep="\t", index=False)

## Fraud Accounts Invoieces:  descriptions, address

In [5]:
sql_fraud_accounts_invoice_descriptions = '''SELECT
                                               systemid,
                                               invoiceid,
                                               description,
                                               notes,
                                               terms,
                                               address
                                            FROM coalesced_live_shards.invoice_stable
                                            WHERE systemid IN (
                                                SELECT systemid
                                                FROM data_science.fraud_accounts_list
                                                );'''

df_fraud_accounts_invoice_descriptions = pd.read_sql_query(sql_fraud_accounts_invoice_descriptions, connect_to_db)

In [6]:
df_fraud_accounts_invoice_descriptions.tail()

,systemid,invoiceid,description,notes,terms,address
262081,729391,1652888,10 Reviews A Month,,VERY IMPORTANT:\r\n1. The payment is Net10 and...,
262082,1762123,2397848,Premium Online Business Listing,,Thank you for your business. Please send payme...,
262083,426063,7119784,Digital Media - Seth Rudetsky Series - Jessie ...,,Please make checks payable to:\r\nCapacity Int...,
262084,235328,1316714,http://www.miamipremiersoccer.net\r\nUpdate of...,**This does NOT guarantee there will be no hac...,Billing & Payment Terms:\r\n50% of estimated g...,
262085,844420,2679132,Search Engine Optimization(Basic),,Net 10,


In [7]:
# Export as csv file
df_fraud_accounts_invoice_descriptions.to_csv("/Users/dwahid/Documents/GitHub/fraud_detection/data/fraud_accounts_invoice_descriptions.csv", 
                                      sep="\t", index=False)

### Report System: For all accounts

In [ ]:
sql_all_account_report_systems = '''SELECT
                   systemid,
                   business_id,
                   admin_identity_id,
                   is_freshbooks_account_active,
                   is_modern,
                   country_code,
                   country,
                   postal_code,
                   street,
                   mob_phone,
                   bus_phone,
                   admin_fname,
                   admin_lname,
                   business_name,
                   admin_email,
                   signup_datetime,
                   signup_date,
                   signup_ip_address,
                   smux_signup,
                   merged_channel_category_1,
                   client_count
                FROM report_systems
                WHERE signup_date BETWEEN '2018-08-01' and '2019-07-30';'''

df_all_accounts_report_systems = pd.read_sql_query(sql_all_account_report_systems, connect_to_db)

In [ ]:
df_all_accounts_report_systems.tail()

In [ ]:

import sys
reload(sys)
sys.setdefaultencoding('utf8')

In [ ]:
df_days_to_ban_fraud_account

In [ ]:
df_all_accounts_report_systems.head()

### Invoices: For all accounts

In [ ]:
sql_all_account_invoices = '''SELECT
                                   invoiceid,
                                   systemid,
                                   customerid,
                                   amount,
                                   paid,
                                   description,
                                   date_paid,
                                   create_date,
                                   terms,
                                   payment_status,
                                   created_at
                                FROM coalesced_live_shards.invoice_stable
                                WHERE systemid IN (
                                        SELECT systemid
                                        FROM report_systems
                                        WHERE signup_date BETWEEN '2018-08-01' and '2019-07-30'
                                    );'''

df_all_accounts_invoices = pd.read_sql_query(sql_all_account_invoices, connect_to_db)

In [ ]:
# Export as csv file
df_all_accounts_invoices.to_csv("/Users/dwahid/Documents/GitHub/fraud_detection/data/invoices_all_accounts_aug18_jul19.csv", 
                                      sep="\t", index=False)